# SER-347 - Introdução à Programação para Sensoriamento Remoto

## 	Análise de respostas da vegetação ao longo de período de seca no semi-árido brasileiro 2010 - 2018 

- Hugo Tameirão Seixas
- Vania Maria de Oliveira


# 1. Introdução
O presente trabalho teve como objetivo avaliar a relação entre variáveis de vegetação obtidas dos produtos MODIS (Moderate-Resolution Imaging Spectroradiometer) com umidade do solo obtida do SMOS (Soil Moisture and Ocean Salinity), e precipitação obtida do CHIRPS (Climate Hazards Group InfraRed Precipitation with Station). A área de estudo está inserida no semi-árido brasileiro, na região de Petrolina-PE/Juazeiro-BA.A análise das variáveis será feita pelo o uso de séries temporais, que possibilitarão a observaço da condição da vegetação ao longo de anos com precipitação abaixo da média anual.

Os produtos MODIS que serão utilizados são: MOD11A2.006, MOD15A2H.006, MOD13A3.006,MOD17A2H.006, MCD12Q1.006. Sendo que deles serão extraídos medidas de LST (Land Surface Temperature), LAI (Leaf Area Index), NDVI (Normalized Difference Vegetation Index), EVI (Enchanced Vegetation Index), e GPP (Gross Primary Production).

<img src="Fluxograma.png">

In [ ]:
# =====================================================================================================================
# Import libraries
# =====================================================================================================================

import requests as r
import ftplib
import cgi
import json
import getpass
import os

import time
import progressbar

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
from sklearn.model_selection import StratifiedShuffleSplit
import datetime
import shapely

import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy
import cartopy.io.img_tiles as cimgt
import seaborn as sns

In [ ]:
# =====================================================================================================================
# Part 1:Setup configuration
# =====================================================================================================================
sep = "========================================================================================================================"
print(sep)

#Inicial Recommendations
print("To start the script it is necessary to define the working directory, in which all download files will be stored, "
      "the extent coordinates of the area of study in decimal degrees (WGS84), "
      "as well the username and password to access the ESA and APPEEARS portal in order to download SMOS data.")
print("")
print("IMPORTANT!! - It is is necessary to enable the API options to access APPEEARS data as shown in : " 
      "https://lpdaacsvc.cr.usgs.gov/appeears/help")
print(sep)

#Enter Inicial information: File path, area of interest, username and password on the ESA and Appears websites
try_again = True
while try_again:
    while True:
        try:
            wdir = input("Define working directory: ")  
            os.chdir(wdir)
            break
        except:
            print("Error setting the working directory")

    while True:
        xbl = float(input("Define bottom left longitude: "))
        if (xbl > -180) and (xbl < 180):
            break
        else:
            print("Invalid longitude")
    while True:
        ybl = float(input("Define bottom left latitude: "))
        if (ybl > -90) and (ybl < 90):
            break
        else:
            print("Invalid latitude")
    while True:
        xur = float(input("Define upper right longitude: "))
        if (xur > -180) and (xur < 180) and (xur > xbl):
            break
        else:
            print("Invalid longitude")
    while True:
        yur = float(input("Define upper right latitude: "))
        if (yur > -180) and (yur < 180) and (yur > ybl):
            break
        else:
            print("Invalid latiitude")
            
    print(sep)
    
    #Longitude X list to create rectangle in plot | Latitude Y list to create rectangule in plot
    x = [xbl, xbl, xur, xur, xbl] 
    y = [ybl, yur, yur, ybl, ybl]  
    
    #Load Basemap
    stamen_terrain = cimgt.StamenTerrain() 
    
    #Create figures to show where study area is located
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 2, 1, projection=stamen_terrain.crs)
    ax.set_extent([xbl-10, xur+10, ybl-10, yur+10])
    ax.add_image(stamen_terrain, 8)
    ax.plot(x, y, color='red', linewidth=1, transform=cartopy.crs.PlateCarree())

    ax2 = fig.add_subplot(1, 2, 2, projection=stamen_terrain.crs)
    ax2.set_extent([xbl-1, xur+1, ybl-1, yur+1])
    ax2.add_image(stamen_terrain, 9)
    ax2.plot(x, y, color='red', linewidth=1, transform=cartopy.crs.PlateCarree())
    
    print("Checking setup:")
    print("Working directory set to: ", wdir)
    print("Bottom left longitude = ", xbl)
    print("Bottom left latitude = ", ybl)
    print("Upper right longitude = ", xur)
    print("Upper right latitude = ", yur)
    print(sep)
    print("Creating sample, please wait...")
    plt.show()
    print("Done!")
    retry = input("Do you want to confirm the settings?(y/n)")  # Option to rerun loop
    if retry == "n":
        try_again = True
        print("Resetting setup settings...")
    elif retry == "y":
        try_again = False
        print("Setup settings confirmed!")
    else:
        print("Invalid input. Resetting setup...")
        
    print(sep)

inAreas = input('Set area of interest method (1/2). -Note- Method 1 uses the Land Cover product from MODIS'
                'to generate random points for the 4 biggest land class for the study area. Method 2 requires'
                'a csv file named as "land_use_points" and must be inserted in the working directory. This csv'
                'file must contain columns named LandUse/lat/lon. (Default = 1) ')    
print('Method {0} was selected'.format(inAreas))
print(sep)

print("ESA sign in, if you don't have an account, access: https://eo-sso-idp.eo.esa.int/idp/umsso20/registration")
user_ESA = input("Insert your ESA username: ")
password_ESA = getpass.getpass("Insert your ESA password: ")
print(sep)

print("APPEEARS sign in, if you don't have an account, access: https://lpdaacsvc.cr.usgs.gov/appeears/")
user_APPEEARS = input("Insert your APPEEARS username: ")
password_APPEEARS = getpass.getpass("Insert your APPEEARS password: ")
print(sep)

print("Setup was succesfully done!")
print(sep)

====================================================================================================================

To start the script it is necessary to define the working directory, in which all download files will be stored, the extent coordinates of the area of study in decimal degrees (WGS84), as well the username and password to access the ESA and APPEEARS portal in order to download SMOS data.

IMPORTANT!! - It is is necessary to enable the API options to access APPEEARS data as shown in : https://lpdaacsvc.cr.usgs.gov/appeears/help

====================================================================================================================

Define working directory: /home/hugo/Documents/Projeto

Define bottom left longitude: -40.8 

Define bottom left latitude: -9.65 

Define upper right longitude: -40 

Define upper right latitude: -8.77 

====================================================================================================================

Checking setup:

Working directory set to:  /home/hugo/Documents/Projeto

Bottom left longitude =  -40.8

Bottom left latitude =  -9.65

Upper right longitude =  -40.0

Upper right latitude =  -8.77

====================================================================================================================
Creating sample, please wait...

<img src="StudyArea.png">

Done!

Do you want to confirm the settings?(y/n) y

Setup settings confirmed!

====================================================================================================================

Set area of interest method (1/2). -Note- Method 1 uses the Land Cover product from MODISto generate random points for the 4 biggest land class for the study area. Method 2 requiresa csv file named as "land_use_points" and must be inserted in the working directory. This csvfile must contain columns named LandUse/lat/lon. (Default = 1) 2
Method 2 was selected

====================================================================================================================

ESA sign in, if you don't have an account, access: https://eo-sso-idp.eo.esa.int/idp/umsso20/registration

Insert your ESA username: HugoSeixas

Insert your ESA password: ········

====================================================================================================================

APPEEARS sign in, if you don't have an account, access: https://lpdaacsvc.cr.usgs.gov/appeears/

Insert your APPEEARS username: Htseixas

Insert your APPEEARS password: ········

====================================================================================================================
Setup was succesfully done!

====================================================================================================================

In [ ]:
# =====================================================================================================================
# Part 2: Create subfolders for each data that will be used
# =====================================================================================================================

print(sep)
print("Creating sub folders and setting paths...")

# Create SMOS directory if not exist
sdir = os.path.join(wdir, "smos") 
if not os.path.exists(sdir):
    os.makedirs(sdir)
print(sdir)

# Create CHIRPS directory if not exist
cdir = os.path.join(wdir, "chirps") 
if not os.path.exists(cdir):
    os.makedirs(cdir)
print(cdir)

# Create MODIS directory if not exist
mdir = os.path.join(wdir, "modis") 
if not os.path.exists(mdir):
    os.makedirs(mdir)
print(mdir)

print("Sub folders created successfully!")
print(sep)

====================================================================================================================
Creating sub folders and setting paths...

/home/hugo/Documents/Projeto/smos

/home/hugo/Documents/Projeto/chirps

/home/hugo/Documents/Projeto/modis

Sub folders created successfully!

====================================================================================================================

In [ ]:
# ===========================================================================================================================================
# Part 3: Part 3 is subdivided into 3.1, 3.2 and 3.3 and correspond to the downloads and 
# processing of each product used in this work 
# ===========================================================================================================================================
# =====================================================================================================================
# Part 3.1 - Data Download SMOS
# =====================================================================================================================

# SMOS
print(sep)
print("Starting to download SMOS Soil Moisture data...")
print(sep)

# Date to start and finish the download. SMOS product to download
begin = "2010-06-01"  
end = "2018-12-31" 
prod = "MIR_SMUDP2"  

print("Start date: ", begin)
print("End date date: ", end)
print("Product: ", prod)
print(sep)

#Download SMOS data
print("Downloading secp utility...")
os.system('wget -P ' + sdir + ' -nc  https://github.com/spinto/secp/raw/master/secp')  # Download secp script
os.system('chmod +x ' + sdir + '/secp')  # Give execution permissions to secp script
print("Done")
print(sep)

# Loop to download files from ascending orbit
for o in ["ASCENDING"]:  
    
    print("Downloading url list of {0} files ...".format(o))
    cmd1 = ('curl --data "service=SimpleOnlineCatalogue&version=1.0&'
           'request=search&format=text%2Fplain&pageCount=10&'
           'query.nativeProductFormat=NetCDF+Format&'
           'query.footprint.minlat={0}&'.format(ybl) +
           'query.footprint.minlon={0}&'.format(xbl) +
           'query.footprint.maxlat={0}&'.format(yur) +
           'query.footprint.maxlon={0}&'.format(xur) +
           'query.beginAcquisition.start={0}&'.format(begin) +
           'query.beginAcquisition.stop={0}&'.format(end) +
           'query.orbitDirection={0}&'.format(o) +
           'query.productType={0}"'.format(prod) +
           ' https://smos-diss.eo.esa.int/socat/SMOS_Open/search > {0}'.format(os.path.join(sdir, "file_url")))
    os.system(cmd1)  # Download url list in txt from a query
    print("Done")
    
    #Create MODIS directory if not exist
    print("Downloading {0} files ...".format(o))
    odir = os.path.join(sdir, o) 
    # Create sub folders to store files of each orbit direction
    if not os.path.exists(odir): 
        os.makedirs(odir)
    cmd2 = ('{0}'.format(os.path.join(sdir, "secp")) +  # Run secp script to download files
            ' -F {0}' .format(os.path.join(sdir, "file_url")) +  # Take txt file with list of url for download
            ' -o {0} -C {1}:{2}'.format(odir, user_ESA, password_ESA) +  # Set local dir and user/password
            ' -P 4') # Parallel workers
    os.system (cmd2) # Download files
    print("Done")

# Delete secp script and txt file
os.remove('{0}'.format(os.path.join(sdir, "secp")))  
os.remove('{0}'.format(os.path.join(sdir, "file_url")))

# Delete user informations
del user_ESA, password_ESA 

print("Download was successfull!")
print(sep)

====================================================================================================================
Starting to download SMOS Soil Moisture data...

====================================================================================================================

Start date:  2010-06-01

End date date:  2018-12-31

Product:  MIR_SMUDP2

====================================================================================================================
Downloading secp utility...

Done

====================================================================================================================

Downloading url list of ASCENDING files ...

Done

Downloading ASCENDING files ...

Done

Download was successfull!

====================================================================================================================

In [ ]:
# =====================================================================================================================
# Part 3.2 - Data Download CHIRPS
# =====================================================================================================================

#Access to UCSB FTP for download
print(sep)
print('Starting to download CHIRPS Rainfall data...')
print('Connecting to ftp access...')
ftp = ftplib.FTP('ftp.chg.ucsb.edu')
ftp.login()
ftp.cwd('/pub/org/chg/products/CHIRPS-2.0/global_monthly/netcdf/')  
flist = ftp.nlst()
file = flist[0]
size = ftp.size(file)

# Set directory and file to receive downloaded data
host = os.path.join(cdir, file)  
local = open(host, 'wb')

print('Download started...')

#Progress bar components and configuration
widgets = ['Downloading: ', progressbar.Percentage(), ' ',
           progressbar.Bar(marker='#',left='[',right=']'),
           ' ', progressbar.ETA(), ' ', progressbar.FileTransferSpeed()]  

pbar = progressbar.ProgressBar(widgets = widgets, maxval = size)  
pbar.start()

# Define function to download data using progress bar
def file_write(data):  
    local.write(data)
    global pbar
    pbar += len(data)

ftp.retrbinary('RETR ' + file, file_write)  

# Close connection
local.close()
ftp.quit()  

print("Download was successfull!")
print(sep)

====================================================================================================================
Starting to download CHIRPS Rainfall data...

Connecting to ftp access...

Download started...
<img src="Progress.png">

Download was successfull!

====================================================================================================================

In [ ]:
# ===================================================================================================================
# Parte 3.3: Download and processing of MODIS data
# ===================================================================================================================

#Access to data will be done through appeears
print(sep)
print('Starting to download MODIS data...')
print('Connecting to APPEEARS API...')
api = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'

#We will use the previously entered username and password.
token_response = r.post('{}login'.format(api), auth=(user_APPEEARS, password_APPEEARS)).json()

del user_APPEEARS, password_APPEEARS

 # MODIS products to be downloaded
print('Setting products to download...')
prods = ['MOD11A2.006', 'MOD15A2H.006', 'MOD13A3.006',
         'MOD17A2H.006', 'MCD12Q1.006'] 

# MODIS layers to be downloaded
print('Setting layers to download...')
layers = [(prods[0], 'LST_Day_1km'), (prods[1], 'Lai_500m'),
          (prods[2], '_1_km_monthly_NDVI'), (prods[2], '_1_km_monthly_EVI'),
          (prods[3], 'Gpp_500m'), (prods[4], 'LC_Type1')]  

prodLayer = []
for l in layers:
    prodLayer.append({"layer": l[1], "product": l[0]})

token = token_response['token']
head = {'Authorization': 'Bearer {}'.format(token)}

# Set area for download based on giver coordinates
print('Setting area to download...')
study_area = shapely.geometry.box(xbl, ybl, xur, yur)  
crs = {'init': 'epsg:4326'}
study_area = gpd.GeoDataFrame(index=[0], geometry=[study_area], crs=crs)
study_area = study_area[0:].to_json()
study_area = json.loads(study_area)

task_name = 'ser_347'
task_type = 'area'
proj = 'geographic'
outFormat = "netcdf4"
startDate = "06-01-2010"
endDate = '12-31-2018'
recurring = False

print('Creating new task...')
task = {'task_type': task_type, 'task_name': task_name,
        'params': {'dates': [{'startDate': startDate, 'endDate': endDate}],
                   'layers': prodLayer,
                   'output': {'format': {'type': outFormat}, 'projection': proj},
                   'geo': study_area}}

task_response = r.post('{}task'.format(api), json=task, headers=head).json()

params = {'limit': 2, 'pretty': True}

tasks_response = r.get('{}task'.format(api), params=params, headers=head).json()

task_id = task_response['task_id']
status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json()

print('Waiting for task processing at APPEEARS...')

#Check task status for every 5 minutes
starttime = time.time()
while r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'] != 'done':
    print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])
    time.sleep(300.0 - ((time.time() - starttime) % 300.0)) 
print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])

bundle = r.get('{}bundle/{}'.format(api, task_id)).json()

print('Starting download...')
files = {}
for f in bundle['files']:
    files[f['file_id']] = f['file_name']
for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), stream=True)
    filename = os.path.basename(cgi.parse_header(dl.headers['Content-Disposition'])[1]['filename'])
    filepath = os.path.join(mdir, filename)
    with open(filepath, 'wb') as f:
        for data in dl.iter_content(chunk_size=8192):
            f.write(data)

print("Download was successfull!")
print(sep)

====================================================================================================================
Starting to download MODIS data...

Connecting to APPEEARS API...

Setting products to download...

Setting layers to download...

Setting area to download...

Creating new task...

Waiting for task processing at APPEEARS...

processing

processing

processing

done

Starting download...

Download was successfull!

====================================================================================================================

In [ ]:
# =====================================================================================================================
# Part 4: Insert regions of interest
# =====================================================================================================================

print(sep)
print('Setting sample for data analysis...')
print(sep)

if inAreas == '1':
    print('Openning MODIS Land Cover product...')
    nc = xr.open_dataset(os.path.join(mdir, 'MCD12Q1.006_500m_aid0001.nc'))
    nc = nc.isel(time=[0])
    nc = nc.drop(['crs', 'QC'])
    numClasses = (len(np.unique(nc['LC_Type1'])))
    
    print('Converting data from NetCDF to Data Frame...')
    df = nc['LC_Type1'].to_dataframe()

    mCLasses = {0:'Water', 1:'Evergreen Needleleaf Forest', 2:'Evergreen Broadleaf Forest',
                3:'Deciduous Needleleaf Forest', 4:'Deciduous Broadleaf Forest',
                5:'Mixed Forests', 6:'Closed Shrublands',
                7:'Open Shrublands', 8:'Woody Savannas',
                9:'Savannas', 10:'Grasslands',
                11:'Permanent Wetlands', 12:'Croplands',
                13:'Urban and Build-up', 14:'Cropland/Natural Vegetation Mosaic',
                15:'Snow and Ice', 16:'Barren or Sparsely Vegetated'}
    colorPalette = {'Water':'#001146', 'Evergreen Needleleaf Forest':'#033500',
                    'Evergreen Broadleaf Forest':'#06470c', 'Deciduous Needleleaf Forest':'#3f9b0b',
                    'Deciduous Broadleaf Forest':'#137e6d', 'Mixed Forests':'#40a368',
                    'Closed Shrublands':'#e6daa6', 'Open Shrublands':'#ad8150', 
                    'Woody Savannas':'#ceb301', 'Savannas':'#9aae07',
                    'Grasslands':'#c7fdb5', 'Permanent Wetlands':'#06c2ac',
                    'Croplands':'#653700', 'Urban and Build-up':'#516572', 
                    'Cropland/Natural Vegetation Mosaic':'#a83c09',
                    'Snow and Ice':'#ffffff', 'Barren or Sparsely Vegetated':'#e2ca76'}

    colorMap = mpl.colors.ListedColormap(['#001146', '#033500', '#06470c', '#3f9b0b', '#137e6d', '#40a368',
                                          '#e6daa6', '#ad8150', '#ceb301', '#9aae07', '#c7fdb5', '#06c2ac',
                                          '#653700', '#516572', '#a83c09', '#ffffff', '#e2ca76'])
    colorNorm = mpl.colors.Normalize(vmin=0,vmax=16)
    
    print(sep)
    df['LandUse'] = df['LC_Type1'].map(mCLasses)
    print(df.head())
    df = df.reset_index()
    print(sep)
    
    print('Plotting land cover map and size of each class...')
    nc['LC_Type1'].plot(add_colorbar=False, cmap=colorMap, norm=colorNorm, size=5)
    patchList = []
    for key in colorPalette:
        data_key = mpl.patches.Patch(color=colorPalette[key], label=key)
        patchList.append(data_key)
    plt.legend(handles=patchList, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title('Land Cover Map')
    plt.ylabel('Latitude')
    plt.xlabel('Longitude')
    plt.show()

    sns.countplot(data=df, y='LandUse', palette=colorPalette)
    plt.ylabel('')
    plt.xlabel('Count')
    plt.show()
    print(sep)
    
    # Remove unwanted classes
    print('Removing unwanted classes...')
    for i in [0, 13, 15, 254, 255]: 
        dfr = df[df['LC_Type1'] != i]
        df = dfr   

    counts = dfr['LC_Type1'].value_counts()
    nClasses = (counts.index.tolist())
    counts = counts.nlargest(4)
    lClasses = (counts.index.tolist())
    nClasses = list(set(nClasses) - set(lClasses))

    # Remove unwanted classes
    print('Filtering 4 biggest classes for analysis...')
    for i in nClasses: 
        dfr = dfr[dfr['LC_Type1'] != i]
    print('Selected classes for analysis: ', dfr['LandUse'].unique().tolist())
    
    print('Creating stratified random sample for each class...')
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.01, random_state=1)

    for train_index, test_index in split.split(dfr, dfr['LC_Type1']):
        sample = dfr.reindex(test_index)
        sample = sample.dropna()
    print('Sample successfully generated!')
    print(sep)
        
elif inAreas == '2':
    print('Loading table with points of interest...')
    land_use_points = pd.read_csv(os.path.join(wdir, 'land_use_points.csv'))
    geometry = [shapely.geometry.Point(xy) for xy in zip(land_use_points.lon, land_use_points.lat)]
    crs = {"init": "epsg:4326"}
    sample = gpd.GeoDataFrame(land_use_points, crs=crs, geometry=geometry)
    print('Sample successfully generated!')
    print(sep)

====================================================================================================================
Setting sample for data analysis...

====================================================================================================================
Loading table with points of interest...

<img src="Coords.png">

Sample successfully generated!

====================================================================================================================

In [ ]:
# =====================================================================================================================
# Part 5 - Part 5 is subdivided into 5.1, 5.2 and 5.3 and opens the netCDF files and extracts the values 
# for the data frames
# =====================================================================================================================
# =====================================================================================================================
# Part 5.1 - Open netCDF files and extract values to data frames - SMOS
# =====================================================================================================================

print(sep)
print('Processing SMOS data...')
for orbit in ["ASCENDING"]:
    print(sep)
    print('Processing {0} data...'.format(orbit))
    dfs = []
    smdir = os.path.join(sdir, orbit)
    flist = os.listdir(smdir)
    for file in flist:
        nc = xr.open_dataset(os.path.join(smdir, file))

        time = file[19:25]
        time = datetime.datetime.strptime(time, '%Y%m')
        time = xr.DataArray(time)

        nc = nc.where((nc.Longitude >= xbl) & (nc.Latitude >= ybl) & (nc.Longitude <= xur) & (nc.Latitude <= yur))
        nc = nc.__getitem__(["Soil_Moisture"])

        print('Calculating mean soil moisture for area of study')
        nc = nc.mean(dim=('n_grid_points'), skipna=True)
        nc['Time'] = time
        nc = nc.expand_dims("")

        df = nc.to_dataframe()
        dfs.append(df)

    dfs = pd.concat(dfs)
    dfs.index = pd.to_datetime(dfs['Time'])
    print('Filling gaps...')
    dfs["Soil_Moisture"].interpolate(method='time', inplace=True)
    print('Calculating monthly mean values...')
    dfs = dfs.resample('M').mean()
    dfs['Time'] = pd.to_datetime(dfs.index)
    dfs.index = pd.to_datetime(dfs["Time"], format='%Y%m').apply(lambda x: x.strftime('%Y-%m'))
    del dfs['Time']
    print('Done!')
    print(sep)
    print(dfs.head())
print(sep)
print('Processing complete!')
print(sep)

====================================================================================================================
Processing SMOS data...

====================================================================================================================
Processing ASCENDING data...

Filling gaps...

Calculating monthly mean values...

Done!

====================================================================================================================
<img src="SoilMoisture.png">

====================================================================================================================
Processing complete!

====================================================================================================================

In [ ]:
# =====================================================================================================================
# Part 5.2 - Open netCDF files and extract values to data frames - CHIRPS
# =====================================================================================================================

print(sep)
print('Processing CHIRPS data...')
print(sep)

# Open file
nc = xr.open_dataset(filename_or_obj=(cdir + '/chirps-v2.0.monthly.nc'))  

print('Slicing data...')
ncs = nc.sel(time=slice('2010-06-01', '2018-12-31'), longitude=slice(xbl, xur), latitude=slice(ybl, yur))

print('Calculating mean monthly rainfall for study area...')
ncs_m = ncs.mean(dim=('latitude', 'longitude'))

print('Converting from NetCDF to Data Frame...')
dfc = ncs_m.to_dataframe()

dfc["Time"] = dfc.index
dfc.index = pd.to_datetime(dfc["Time"], format='%Y%m').apply(lambda x: x.strftime('%Y-%m'))
del dfc["Time"]

print('Done!')
print(sep)
print(dfc.head())
print(sep)
print('Processing complete!')
print(sep)

====================================================================================================================
Processing CHIRPS data...

====================================================================================================================
Slicing data...

Calculating mean monthly rainfall for study area...

Converting from NetCDF to Data Frame...

Done!

====================================================================================================================
<img src="Precip.png">

====================================================================================================================
Processing complete!

====================================================================================================================

In [ ]:
# =====================================================================================================================
# Part 5.3 - Open netCDF files and extract values to data frames - MODIS
# =====================================================================================================================

print(sep)
print('Processing MODIS data...')
print(sep)

layers = ['LST_Day_1km', 'Lai_500m', '_1_km_monthly_NDVI', '_1_km_monthly_EVI', 'Gpp_500m']

flist = []
for file in os.listdir(mdir):
    if file.endswith(".nc"):
        flist.append(file)
flist.remove("MCD12Q1.006_500m_aid0001.nc")

dfm = []

print('Calculating monthly mean values of LST, LAI, NDVI, EVI and GPP for each land cover/points of interest...')
row = next(sample.iterrows())
for f in flist:
    print('Processing {0} data...'.format(f))
    nc = xr.open_dataset(filename_or_obj=(os.path.join(mdir, f)))
    for c, row in sample.iterrows():
        nvar = nc.sel(lon=row['lon'], lat=row['lat'], method='nearest')
        df = nvar.to_dataframe()
        df.index = df.index.to_datetimeindex()
        for l in layers:
            try:
                df[l].interpolate(method='time', inplace=True)
            except:
                pass
        df = df.resample('M').mean()
        df['Time'] = df.index
        df.index = pd.to_datetime(df["Time"], format='%Y%m').apply(lambda x: x.strftime('%Y-%m'))
        df['Time'] = df.index
        df['LandUse'] = row['LandUse']
        df.set_index(['Time', 'LandUse'], inplace=True)
        dfm.append(df)

print('Merging Data Frames...')
dfm = pd.concat(dfm, axis=1).sort_index(axis=1)
print('Filtering Data Frames...')
dfm = dfm[layers]
dfm = dfm.groupby(level=0, axis=1).mean()
dfm = dfm.dropna()
print('Done!')
print(sep)
print(dfm.head())
print(sep)
print('Processing complete!')
print(sep)

====================================================================================================================
Processing MODIS data...

====================================================================================================================
Calculating monthly mean values of LST, LAI, NDVI, EVI and GPP for each land cover/points of interest...

Processing MOD11A2.006_1km_aid0001.nc data...

Processing MOD17A2H.006_500m_aid0001.nc data...

Processing MOD16A2.006_500m_aid0001.nc data...

Processing MOD13A3.006_1km_aid0001.nc data...

Processing MOD15A2H.006_500m_aid0001.nc data...

Merging Data Frames...

Filtering Data Frames...

Done!

====================================================================================================================
<img src="MODIS.png">

====================================================================================================================
Processing complete!

====================================================================================================================

In [ ]:
print(sep)
print('Merging Data Frames...')
print(sep)

df = dfm.join(dfc, how='right')
df = df.join(dfs, how='right')
colnames = ['GPP', 'LST', 'LAI', 'EVI', 'NDVI', 'PRECIP', 'SM']
df.columns = colnames
df.index = df.index.set_levels([pd.to_datetime(df.index.levels[0]), df.index.levels[1]])
df.to_csv(r'' + wdir + '/df' + '.csv')

print('Done!')
print(sep)
print(df.head())
print(sep)

====================================================================================================================
Merging Data Frames...

====================================================================================================================
Done!

====================================================================================================================
<img src="Merge.png">

====================================================================================================================

In [ ]:
# =====================================================================================================================
# Part 6 - Data analysis
# =====================================================================================================================

#Generating time series
print(sep)
print('Generating time series...')
print(sep)

dfu = df.unstack()

fig, axes = plt.subplots(nrows=7, ncols=1, figsize=(20,20), sharex=True)

dfu.GPP.plot(ax=axes[0], title='GPP', legend=False)
dfu.LAI.plot(ax=axes[1], title='LAI', legend=False)
dfu.EVI.plot(ax=axes[2], title='EVI', legend=False)
dfu.NDVI.plot(ax=axes[3], title='NDVI', legend=False)
dfu.LST.plot(ax=axes[4], title='LST', legend=False)
dfu.PRECIP.plot(ax=axes[5], title='PRECIP', legend=False)
dfu.SM.plot(ax=axes[6], title='SM', legend=False)

lg = axes[0].legend(bbox_to_anchor=(1.02, 1), loc=2, ncol=1)

fig.tight_layout()
plt.show()
print(sep)

====================================================================================================================
Generating time series...

====================================================================================================================
<img src="TimeSeries.png">

====================================================================================================================

In [ ]:
#Generating time series tendency
print(sep)
print('Generating time series tendency...')
print(sep)

dfr = dfu[colnames].rolling(window=12, center=True).mean()

fig, axes = plt.subplots(nrows=7, ncols=1, figsize=(20,20), sharex=True)

dfr.GPP.plot(ax=axes[0], title='GPP', legend=False)
dfr.LAI.plot(ax=axes[1], title='LAI', legend=False)
dfr.EVI.plot(ax=axes[2], title='EVI', legend=False)
dfr.NDVI.plot(ax=axes[3], title='NDVI', legend=False)
dfr.LST.plot(ax=axes[4], title='LST', legend=False)
dfr.PRECIP.plot(ax=axes[5], title='PRECIP', legend=False)
dfr.SM.plot(ax=axes[6], title='SM', legend=False)

lg = axes[0].legend(bbox_to_anchor=(1.02, 1), loc=2, ncol=1)

fig.tight_layout()
plt.show()
print(sep)

====================================================================================================================
Generating tendency series...

====================================================================================================================
<img src="Tendency.png">

====================================================================================================================

In [ ]:
#Graphics generation
print(sep)
print('Generating scatter plot between Soil Moisture and MODIS products...')
print(sep)

dfp = df.reset_index()

sns.lmplot(x="SM", y='GPP', hue="LandUse", legend='full', data=dfp, height=4)
sns.lmplot(x="SM", y='LAI', hue="LandUse", legend='full', data=dfp, height=4)
sns.lmplot(x="SM", y='EVI', hue="LandUse", legend='full', data=dfp, height=4)
sns.lmplot(x="SM", y='NDVI', hue="LandUse", legend='full', data=dfp, height=4)
sns.lmplot(x="SM", y='LST', hue="LandUse", legend='full', data=dfp, height=4)
plt.show()
print(sep)

====================================================================================================================
Generating scatter plot between Soil Moisture and MODIS products...

====================================================================================================================
<img src="SM1.png">
<img src="SM2.png">
<img src="SM3.png">
<img src="SM4.png">
<img src="SM5.png">

====================================================================================================================

In [ ]:
#Graphics generation
print(sep)
print('Generating scatter plot between Precipitation and MODIS products...')
print(sep)

sns.lmplot(x="PRECIP", y='GPP', hue="LandUse", legend='full', data=dfp, height=4)
sns.lmplot(x="PRECIP", y='LAI', hue="LandUse", legend='full', data=dfp, height=4)
sns.lmplot(x="PRECIP", y='EVI', hue="LandUse", legend='full', data=dfp, height=4)
sns.lmplot(x="PRECIP", y='NDVI', hue="LandUse", legend='full', data=dfp, height=4)
sns.lmplot(x="PRECIP", y='LST', hue="LandUse", legend='full', data=dfp, height=4)
plt.show()
print(sep)

====================================================================================================================
Generating scatter plot between Precipitation and MODIS products...

====================================================================================================================
<img src="P1.png">
<img src="P2.png">
<img src="P3.png">
<img src="P4.png">
<img src="P5.png">

====================================================================================================================